In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import os

In [ ]:
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )


In [ ]:
# Run app and display result inline in the notebook
app.run_server(mode='external', port=8100, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [5]:
os.environ["CDSW_ENGINE_ID"]+"."+os.environ["CDSW_DOMAIN"]

'7uehs8jzathsicxm.ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site'

In [7]:
JupyterDash.infer_jupyter_proxy_config()

QueueEmpty: 